## Training Model

### Loading packages

In [90]:
import numpy as np
import pandas as pd
import pdb

from deepiv.models import Treatment, Response
import deepiv.architectures as architectures
import deepiv.densities as densities

import tensorflow as tf

from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras.layers.merge import Concatenate
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras import regularizers, optimizers
from sklearn.model_selection import train_test_split

seed = 123

### Importing data

In [2]:
expression_levels_raw = pd.read_csv('/Users/billyf/deepiv-gwas/derived_data/expression_levels_subset.csv')
gene_variants_raw = pd.read_csv('/Users/billyf/deepiv-gwas/derived_data/gene_variants.csv')
outcomes_raw = pd.read_csv('/Users/billyf/deepiv-gwas/derived_data/outcomes.csv')

In [3]:
# sanity check
assert(gene_variants_raw.shape[1] - 1 == outcomes_raw.shape[0])
print("Dim(z) = ", gene_variants_raw.shape)
print("Dim(p) = ", expression_levels_raw.shape)
print("Dim(y) = ", outcomes_raw.shape)

Dim(z) =  (13980, 12667)
Dim(p) =  (2344, 12667)
Dim(y) =  (12666, 2)


In [4]:
expression_levels = expression_levels_raw.drop('mrna', axis = 1).transpose().as_matrix()
gene_variants = gene_variants_raw.drop('hugo', axis = 1).transpose().as_matrix()
outcomes = outcomes_raw['outcome'].as_matrix()

In [5]:
print("Dim(z) = ", gene_variants.shape)
print("Dim(p) = ", expression_levels.shape)
print("Dim(y) = ", outcomes.shape)

Dim(z) =  (12666, 13980)
Dim(p) =  (12666, 2344)
Dim(y) =  (12666,)


In [186]:
z_train, z_test, p_train, p_test, y_train, y_test = train_test_split(gene_variants, expression_levels, outcomes, test_size=0.1, random_state=seed)

### Training first stage model

In [6]:
n = len(outcomes)
epochs = int(1500000./float(n)) # heuristic to get epochs
batch_size = 100

instruments = Input(shape=(gene_variants.shape[1],), name = "instruments")
hidden = [7000, 4000, 3000]


activation = "relu"

n_components = expression_levels.shape[1]

est_treat = architectures.feed_forward_net(instruments,
                                          lambda x: densities.mixture_of_gaussian_output(x, n_components),
                                          hidden_layers = hidden,
                                          activations = activation)

treatment_model = Treatment(inputs=[instruments], outputs=est_treat)
print("Fitting treatment")
treatment_model.compile('adam',
                       loss='mixture_of_gaussians',
                       n_components = n_components)

treatment_model.fit(gene_variants, expression_levels,
                   epochs = epochs,
                   batch_size = batch_size)

Fitting treatment
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/118


ValueError: Cannot feed value of shape (100, 2344) for Tensor 'concatenate_1_target:0', which has shape '(?, 1)'

In [94]:
n = len(outcomes)
epochs = int(1500000./float(n)) # heuristic to get epochs
batch_size = 100

instruments = Input(shape=(gene_variants.shape[1],), name = "instruments")
hidden = [100, 100, 100, 100, 100, 100]


activation = "relu"
l2_reg = 0.0001
w_reg = regularizers.l2(l2_reg)

n_components = expression_levels.shape[1]

firstStage = Sequential([
    Dense(hidden[0], activation='relu', input_dim = 13980, name='fc1'),
    Dense(hidden[1], activation='relu', name = 'fc2'),
    Dense(hidden[2], activation='relu', name = 'fc3'),
    Dense(hidden[3], activation='relu', name = 'fc4'),
    Dense(hidden[4], activation='relu', name = 'fc5'),
    Dense(hidden[5], activation='relu', name = 'fc6'),
    Dense(2344, activation = 'relu', name = 'output')])

#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon = 1e-08)
    
firstStage.compile(optimizer='adam', loss='mse')


firstStage.fit(z_train, p_train, epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/118
11399/11399 [==============================] - 8s - loss: 1333658.1120     
Epoch 2/118
11399/11399 [==============================] - 7s - loss: 1283039.6250     
Epoch 3/118
11399/11399 [==============================] - 8s - loss: 1048776.2033     
Epoch 4/118
11399/11399 [==============================] - 8s - loss: 693894.6032     
Epoch 5/118
11399/11399 [==============================] - 9s - loss: 481639.6994     
Epoch 6/118
11399/11399 [==============================] - 9s - loss: 410250.6696     
Epoch 7/118
11399/11399 [==============================] - 8s - loss: 368132.4292     
Epoch 8/118
11399/11399 [==============================] - 8s - loss: 365809.8767     
Epoch 9/118
11399/11399 [==============================] - 7s - loss: 349601.0841     
Epoch 10/118
11399/11399 [==============================] - 7s - loss: 332854.9230     
Epoch 11/118
11399/11399 [==============================] - 8s - loss: 329669.9657     
Epoch 12/118
11399/11399 [============

11399/11399 [==============================] - 8s - loss: 252252.0695     
Epoch 95/118
11399/11399 [==============================] - 7s - loss: 249591.2852     
Epoch 96/118
11399/11399 [==============================] - 7s - loss: 255568.4739     
Epoch 97/118
11399/11399 [==============================] - 7s - loss: 251696.2415     
Epoch 98/118
11399/11399 [==============================] - 7s - loss: 251487.3969     
Epoch 99/118
11399/11399 [==============================] - 7s - loss: 249231.7636     
Epoch 100/118
11399/11399 [==============================] - 8s - loss: 249042.2975     
Epoch 101/118
11399/11399 [==============================] - 7s - loss: 248302.2286     
Epoch 102/118
11399/11399 [==============================] - 7s - loss: 246601.3277     
Epoch 103/118
11399/11399 [==============================] - 7s - loss: 248034.2970     
Epoch 104/118
11399/11399 [==============================] - 7s - loss: 248524.1815     
Epoch 105/118
11399/11399 [=============

In [196]:
n = len(outcomes)
epochs = int(1500000./float(n)) # heuristic to get epochs
batch_size = 100

mrna = Input(shape=(expression_levels.shape[1],), name = "mrna")
hidden = [100, 50]


activation = "relu"
l2_reg = 0.0001
w_reg = regularizers.l2(l2_reg)

n_components = expression_levels.shape[1]

p_hat_train = firstStage.predict(z_train)


secondStage = Sequential([
    Dense(hidden[0], activation='tanh', input_dim = 2344, name='fc1'),
    Dense(hidden[1], activation = 'tanh', name = 'fc2'),
    Dense(1, activation = 'sigmoid', name = 'output')])
    
secondStage.compile(optimizer='adam', loss='binary_crossentropy')





IndexError: list index out of range

In [178]:
secondStage.fit(p_hat_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/118
11399/11399 [==============================] - 1s - loss: 0.3153     
Epoch 2/118
11399/11399 [==============================] - 1s - loss: 0.2538     
Epoch 3/118
11399/11399 [==============================] - 0s - loss: 0.2545     
Epoch 4/118
11399/11399 [==============================] - 1s - loss: 0.2559     
Epoch 5/118
11399/11399 [==============================] - 1s - loss: 0.2564     
Epoch 6/118
11399/11399 [==============================] - 1s - loss: 0.2560     
Epoch 7/118
11399/11399 [==============================] - 1s - loss: 0.2559     
Epoch 8/118
11399/11399 [==============================] - 1s - loss: 0.2559     
Epoch 9/118
11399/11399 [==============================] - 1s - loss: 0.2560     
Epoch 10/118
11399/11399 [==============================] - 1s - loss: 0.2560     
Epoch 11/118
11399/11399 [==============================] - 1s - loss: 0.2561     
Epoch 12/118
11399/11399 [==============================] - 0s - loss: 0.2559     
Epoch 13/118


11399/11399 [==============================] - 1s - loss: 0.2559     
Epoch 100/118
11399/11399 [==============================] - 0s - loss: 0.2558     
Epoch 101/118
11399/11399 [==============================] - 0s - loss: 0.2558     
Epoch 102/118
11399/11399 [==============================] - 1s - loss: 0.2558     
Epoch 103/118
11399/11399 [==============================] - 0s - loss: 0.2559     
Epoch 104/118
11399/11399 [==============================] - 1s - loss: 0.2557     
Epoch 105/118
11399/11399 [==============================] - 1s - loss: 0.2558     
Epoch 106/118
11399/11399 [==============================] - 1s - loss: 0.2558     
Epoch 107/118
11399/11399 [==============================] - 1s - loss: 0.2558     
Epoch 108/118
11399/11399 [==============================] - 0s - loss: 0.2557     
Epoch 109/118
11399/11399 [==============================] - 0s - loss: 0.2557     
Epoch 110/118
11399/11399 [==============================] - 0s - loss: 0.2556     
Epoch 

In [111]:
secondStage.fit(p_train, y_train, epochs = epochs, batch_size = batch_size, verbose = 1)

Epoch 1/118
11399/11399 [==============================] - 1s - loss: 0.1257     
Epoch 2/118
11399/11399 [==============================] - 0s - loss: 0.0226     
Epoch 3/118
11399/11399 [==============================] - 1s - loss: 0.0112     
Epoch 4/118
11399/11399 [==============================] - 1s - loss: 0.0080     
Epoch 5/118
11399/11399 [==============================] - 1s - loss: 0.0059     
Epoch 6/118
11399/11399 [==============================] - 1s - loss: 0.0051     
Epoch 7/118
11399/11399 [==============================] - 1s - loss: 0.0043     
Epoch 8/118
11399/11399 [==============================] - 1s - loss: 0.0037     
Epoch 9/118
11399/11399 [==============================] - 1s - loss: 0.0033     
Epoch 10/118
11399/11399 [==============================] - 1s - loss: 0.0027     
Epoch 11/118
11399/11399 [==============================] - 1s - loss: 0.0021     
Epoch 12/118
11399/11399 [==============================] - 1s - loss: 0.0016     
Epoch 13/118


11399/11399 [==============================] - 0s - loss: 3.6432e-06     
Epoch 96/118
11399/11399 [==============================] - 0s - loss: 3.4412e-06     
Epoch 97/118
11399/11399 [==============================] - 0s - loss: 3.2500e-06     
Epoch 98/118
11399/11399 [==============================] - 0s - loss: 3.0702e-06     
Epoch 99/118
11399/11399 [==============================] - 0s - loss: 2.8995e-06     
Epoch 100/118
11399/11399 [==============================] - 0s - loss: 2.7388e-06     
Epoch 101/118
11399/11399 [==============================] - 0s - loss: 2.5875e-06     
Epoch 102/118
11399/11399 [==============================] - 0s - loss: 2.4441e-06     
Epoch 103/118
11399/11399 [==============================] - 0s - loss: 2.3085e-06     
Epoch 104/118
11399/11399 [==============================] - 0s - loss: 2.1811e-06     
Epoch 105/118
11399/11399 [==============================] - 0s - loss: 2.0601e-06     
Epoch 106/118
11399/11399 [=======================

In [179]:
#Test

p_hat_test = firstStage.predict(z_test)
y_hat_test_deepiv = secondStage.predict(p_hat_test)
y_hat_test_naive = secondStage.predict(p_test)

In [180]:
y_hat_test_deepiv = (y_hat_test_deepiv > .5).astype(int)
y_hat_test_naive = (y_hat_test_naive > .5).astype(int)
y_hat_test_naive = y_hat_test_naive.astype(int)

In [181]:
print(y_test.shape)
print(y_hat_test_deepiv.shape)
print(y_hat_test_naive.shape)

(1267, 1)
(1267, 1)
(1267, 1)


In [187]:
y_test = y_test[:,np.newaxis]
y_test.shape

(1267, 1)

In [188]:
class_error_vec_deepiv = (y_test != y_hat_test_deepiv).astype(int)
class_error_vec_naive= (y_test != y_hat_test_naive).astype(int)

In [195]:
class_error_deepiv = np.mean(class_error_vec_deepiv)
class_error_naive = np.mean(class_error_vec_naive)
print('DeepIV classification error is', class_error_deepiv)
print('OneStage classification error is', class_error_naive)

DeepIV classification error is 0.0718232044198895
OneStage classification error is 0.0055248618784530384
